# Advanced dimensionality reduction techniques

This notebook demonstrates advanced dimensionality reduction techniques such as Principal Component Analysis (PCA), Multi-Dimensional Scaling (MDS), and t-Distributed Stochastic Neighbor Embedding (t-SNE) on both image and text datasets. The project provides a hands-on exploration of how these techniques can be used to reduce high-dimensional data into lower-dimensional spaces while preserving meaningful patterns and structures.

## Outline

In this notebook we will:
* Explain the following advanced dimensionality reduction techniques:
    * Principal component analysis
    * Multi-dimensional scaling
    * t-SNE
* Implement these techniques on an image dataset.
* Implement these techniques on a text dataset.

## Dimensionality reduction on images
This notebook will provide a high-level overview of some of the more advanced techniques for dimensionality reduction. We will be using the [handwritten digits data set](http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits), which is a collection of images of handwritten digits between 0 and 9. The data was generated by a total of 43 people, who wrote a total of 5620 digits by hand which were then digitised and processed into 8x8 greyscale images.

We'll start by importing the necessary packages.

In [ ]:
from time import time
import numpy as np
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, preprocessing)
from matplotlib import offsetbox
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='whitegrid', palette='muted',
        rc={'figure.figsize': (15,10)})

### Loading and inspecting the data
The digits dataset is included as part of the `sklearn` library, which means loading it into the notebook is a breeze. For this train, we will simplify things by only looking at the first six digits in the dataset. We can use the `n_class` argument in the `load_digits()` function to select only the numbers from 0 to 5.

In [ ]:
digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape

print("Total number of samples: ", n_samples)
print("Features per sample: ", n_features)

Below, the image shows a selection from the 64-dimensional digits dataset. Each digit is represented as an 8x8 array of pixels, with values ranging between 0 (white) and 16 (black).

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=10, ncols=10, figsize=(6, 6))
for idx, ax in enumerate(axs.ravel()):
    ax.imshow(X[idx].reshape((8, 8)), cmap=plt.cm.binary)
    ax.axis("off")
_ = fig.suptitle("A selection from the 64-dimensional digits dataset", fontsize=16)

The same concept applies to our dataset, only the arrays have been strung out into single rows of 64 values, ranging between 0 and 16. Let's take a look at the first digit in the data:

In [ ]:
X[0,:]

In [ ]:
y[0]

Could you tell that the first array was a zero? Unless you are some sort of savant genius, you probably wouldn't be able to tell which digit is represented by looking only at the array values.

### Dimensionality reduction techniques
Now what we are going to try and do is reduce the dimensionality of the entire dataset to just two dimensions (down from 64 dimensions – one per pixel). We will use a few different dimensionality reduction techniques, and at each stage, we will plot the data and see how it is distributed in these two dimensions.   

What is very important to note here is that none of these algorithms will be shown the labels of the data; they will be entirely unsupervised. In each case, we will plot the data in two dimensions, but include the known labels of the data points in the plots for our own validation.   

For plotting, we are going to use the same `plot_embedding()` function defined in the original `sklearn` example code. This does a fantastic job of plotting the digits in a two-dimensional space, so there is no need to reinvent the wheel here.   

**Note:** You will see the word **"embedding"** used in some of the comments and plots below. Simply put, an embedding is a representation of a vector in a different feature space. So in this case, the original digits exist as 64-dimensional arrays and are then reduced to just two dimensions. The resulting two-dimensional vectors are referred to as the embedding vectors.

In [ ]:
# Scale and visualise the embedding vectors
def plot_embedding(X, title=None):
    
    # normalise data
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    
    ax = plt.subplot(111)
    
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

### Principal component analysis (PCA)
The objective of PCA is to decompose a dataset into mutually orthogonal components that **each maximise the variance in the dataset**.   

We will use PCA to decompose the dataset into the first two principal components, which will contain the largest and second-largest amounts of variance, respectively.

In [ ]:
print("Computing PCA projection")
t0 = time()
X_pca = decomposition.PCA(n_components=2).fit_transform(X)
t1 = time()
print("Finished PCA projection in " + str(t1-t0) + "s.")

Now, before using the awesome `plot_embedding()` function, let's plot the data, without labels, in the first two principal components.

In [ ]:
ax = sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1],
                     sizes=(10, 200))
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

Can you spot any obvious clusters? Perhaps one or two on the right side of the plot?   

Now, let's plot the data with labels:

In [ ]:
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (t1 - t0))
plt.show()

So we can see that digits do seem to group together, but there isn't the clearest separation between the different digits.   

Now, let's take a look at some other techniques and see how they perform.   

### Multi-dimensional scaling (MDS)
The goal of MDS is to map features to a low-dimensional space **while preserving the distances** between observations in a given dataset.   

MDS can be performed using algorithms that are either *metric* or *non-metric*. Non-metric approaches are typically used to preserve ordinality (order) within data. This is more of a necessity when there are categorical features present in the data. Since the data used here is entirely numeric, we will use a metric approach.   

The **stress** is a measure of the degree to which distances between points in the original feature space correspond with the distances in the low-dimensional space. A lower stress value is preferred, and it is this quantity that is minimised by MDS.

For more information on MDS, read the [`sklearn` user guide](https://scikit-learn.org/stable/modules/manifold.html#multidimensional-scaling).

In [ ]:
print("Computing MDS embedding")
clf = manifold.MDS(n_components=2, 
                   n_init=4, 
                   max_iter=200,
                   n_jobs=-1,
                   random_state=42,
                   dissimilarity='euclidean')
t0 = time()
X_mds = clf.fit_transform(X)
t1 = time()
print("Done. Time: %f" % (t1-t0))
print("Stress: %f" % clf.stress_)

In [ ]:
plot_embedding(X_mds,
               "MDS embedding of the digits (time %.2fs)" %
               (t1 - t0))

here we can see that the separation of the clusters is similar to `PCA` and you can't tell which one is better, so let's continue and try one more technique

### t-distributed stochastic neighbor embedding (t-SNE)
t-SNE is a very complex technique, which can often yield truly stunning results when reducing high-dimensional datasets. Here is a pretty good explanation of t-SNE from [Wikipedia](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding):   
> "The t-SNE algorithm comprises two main stages. First, t-SNE constructs a probability distribution over pairs of high-dimensional objects in such a way that similar objects have a high probability of being picked, whilst dissimilar points have an extremely small probability of being picked. Second, t-SNE defines a similar probability distribution over the points in the low-dimensional map, and it minimises the [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between the two distributions with respect to the locations of the points in the map. Note that whilst the original algorithm uses the Euclidean distance between objects as the base of its similarity metric, this should be changed as appropriate."   

## 🎯 Understanding t-SNE in Simple Terms

t-SNE (t-Distributed Stochastic Neighbor Embedding) is a powerful technique for visualizing high-dimensional data in a lower-dimensional space. Here's how it works step by step:

1. **Measure Similarity** 🔍  
   - Compute how similar each data point is to the others in the original high-dimensional space.

2. **Random Placement** 🎲  
   - Map the data points into a lower-dimensional space (e.g., 2D or 3D) randomly.

3. **Attraction & Repulsion** ⚖️  
   - Similar points pull each other closer.  
   - Dissimilar points push each other apart.

4. **Iterative Refinement** 🔄  
   - Adjust each point’s position step by step until the lower-dimensional representation preserves the original clusters.

👉 The result? A visually meaningful representation where similar data points stay close, making patterns and clusters easier to interpret! 🎨✨


Additionally, you can watch this [video](https://www.youtube.com/embed/NEaUSP4YerM) for a more detailed explanation of how t-SNE works:

> **📝 Note:**  
> t-SNE is **great for visualization** but **not for feature extraction**!  
> Unlike PCA or MDS, t-SNE does **not preserve distances** between points—only their relative similarities.  
> This means you shouldn’t use t-SNE-transformed features for machine learning models. 🚫🤖


In [ ]:
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2,
                     perplexity=40,
                     metric='euclidean',
                     init='pca',
                     verbose=1,
                     random_state=42)
t0 = time()
X_tsne = tsne.fit_transform(X)
t1 = time()
print("Computed t-SNE embedding in %fs." %(t1-t0))

In [ ]:
plot_embedding(X_tsne,
               "t-SNE embedding of the digits (time %.2fs)" %
               (t1 - t0))

plt.show()

Wow, how cool is that?!   

Remember, the algorithm made no use of the labels. The digit arrays were reduced into a two-dimensional space where similar digits ended up close together. We have six main clusters – one for each digit class. But we also have a few smaller clusters – notice the small group of twos that are somewhere between the 1s and the rest of the 2s? Or the 1s that are quite close to the 2s?   

### Word of caution in t-SNE
In this case, we have demonstrated the power of t-SNE as a tool for exploratory data analysis and to reveal natural groups, or clusters, within datasets.   

However, t-SNE can also be very misleading at times and you are encouraged to investigate the effects of the various hyperparameters of the results when working with different datasets.   

Check out [this article](https://distill.pub/2016/misread-tsne/) for some insights into t-SNE's various hyperparameters.